In [4]:
import numpy as np
import scipy.ndimage as snd


In [2]:
# PVs for OTR2 

# get flattened image array
take_image = 'OTRS:IN20:571:IMAGE' 

# need to reshape(n_col,n_row)
ncol  = 'OTRS:IN20:571:N_OF_COL'
n_row = 'OTRS:IN20:571:N_OF_ROW'

# [um/pixel] ## ???
resolution = 'OTRS:IN20:571:RESOLUTION' 

# do images need to be flipped?
def flipIm(image):
    return np.fliplr(image)

# another option is opening profMon GUI and reading XMRS/YRMS PVs
# use TRIM command (?) to check if quads are done settling

In [6]:
# JUST TAKES A "PROFILE" aling center and fits Gaussian and takes STD???
# doesn't even work, I think initial guesses fail at fitting (see notebook)

def calculateRMS(image='None'):#Not complete
    '''Calculates x and y rms given a shaped image and um/pixel number; in this application only used for OTR2'''

    #Set a threshold while calculating center of mass
    ycentroid,xcentroid = snd.center_of_mass(image > (image.mean()+4*image.std()))
    print(xcentroid)
    #Take slice of xdata using ycentroid position
    xdataslice=image[int(ycentroid)]
    #Make simple xdata list with values from 0 to 639 (length 640)
    xdata=range(image.shape[1])

    #Fit gaussian using curve_fit, get coefficients back
    coeffx,var_matrixx = curve_fit(gaus,xdata,xdataslice,p0=[1,200,100])
    xfitmean,xstandarddeviation=coeffx[1],abs(coeffx[2])

    ydataslice=[]
    for xline in image: 
        ydataslice.append(xline[int(xcentroid)])
    ydata=range(image.shape[0])

    #Fit gaussian using curve_fit, get coefficients back
    coeffy,var_matrixy = curve_fit(gaus,ydata,ydataslice,p0=[1,200,100])
    yfitmean,ystandarddeviation=coeffy[1],abs(coeffy[2])

    xrms,yrms= xstandarddeviation,ystandarddeviation
    return xrms,yrms

def gaus(x,a,x0,sigma):  
    return a*np.exp(-(x-x0)**2/(2*sigma**2))#Function just to return gaussian form for curve_fit

In [ ]:
# check rms calc with matlab profiles and beamsizes

In [7]:
% Get beam and profile moments w/ area cut.
if any(opts.usemethod == 6)
    beam.rms_cut_area=get_stats('get_rms',profs,opts,[],opts.cut);
end


%--------------------------------------------------------------
function [stat, prof, name, statStd] = get_rms(prof, bg, cut_peak, cut_area)
% Get integrated moments. Cut optional, defaults to no cut

name='RMS';%bg=0;
if nargin < 4, cut_area=[];end
if nargin < 3, cut_peak=[];end

%if fitbg
%    bg=util_bgLevel(prof);
%    [a,b]=hist(prof(2,:),size(prof,2)*2);a(end)=0;
%    par=util_gaussFit(b,a,0,0);
%    if par(2) < min(b)
%        [d,idx]=max(a);
%        par(2)=b(idx);
%    end
%    bg=par(2);
    prof(2,:)=prof(2,:)-bg;
%end

if ~isempty(cut_peak)
    idx=prof(2,:) < cut_peak*max(prof(2,:));
    prof(2,idx)=0;
    name='RMS cut peak';
end

if ~isempty(cut_area)
    int=cumsum(prof(2,:));
    idLow=find(int < cut_area/2*int(end),1,'last');
    idHigh=find(int > (1-cut_area/2)*int(end),1,'first');
    prof(2,[1:idLow idHigh:end])=0;
    name='RMS cut area';
end

[int,xmean,xrms,xvar,skew,kurt]=util_moments(prof(1,:),prof(2,:));
dx=abs(mean(diff(prof(1,:))));
prof(3,:)=prof(1,:)*0+bg;
if xrms ~= 0
    prof(3,:)=prof(3,:)+int*dx/sqrt(2*pi)/xrms*exp(-((prof(1,:)-xmean)/xrms).^2./2);
end
prof(2,:)=prof(2,:)+bg;
stat=[int*dx xmean xrms skew kurt];
statStd=stat*0;


%--------------------------------------------------------------
function data = get_stats(method, profs, opts, varargin)
% Calculate correlation and pack everything into DATA.

[s.x,s.y,s.u,s.xStd,s.yStd,s.uStd]=deal([0 0 0 0 0]);
m='';int=0;intStd=0;
for tag='xyu'
    is.(tag)=isfield(profs,tag);
    if is.(tag)
        [s.(tag),data.(['prof' tag]),m, ...
            s.([tag 'Std'])]=feval(method,profs.(tag),opts.fitbg.(tag),varargin{:});
        dx=1;
        if opts.isimage && size(profs.(tag),2) > 1
            dx=abs(mean(diff(profs.(tag)(1,:))));
        end
        s.(tag)(1)=s.(tag)(1)/dx;
        data.([tag 'Stat'])=s.(tag);
        data.([tag 'StatStd'])=s.([tag 'Std']);
        int=s.(tag)(1);intStd=s.([tag 'Std'])(1);
    end
end

c=1;
if opts.isimage
    c=abs(mean(diff(profs.y(1,:)))/mean(diff(profs.x(1,:))));
end
xy=-s.u(3)^2*2*c/(1+c^2)+s.x(3)^2/2*c+s.y(3)^2/2/c;
dxy=[s.x(3)*c s.y(3)/c -2*s.u(3)*2*c/(1+c^2)];
xyStd=sqrt(diag(dxy*diag([s.xStd(3)^2 s.yStd(3)^2 s.uStd(3)^2])*dxy'))';

if is.x && is.y
    int=real(sqrt(s.x(1)*s.y(1)));
    dint=[sqrt(s.y(1)) sqrt(s.x(1))]/2/(int+1e-20);
    intStd=sqrt(diag(dint*diag([s.xStd(1)^2 s.yStd(1)^2])*dint'))';
end

data.method=m;
data.stats=[s.x(2) s.y(2) s.x(3) s.y(3) xy int];
data.statsStd=[s.xStd(2) s.yStd(2) s.xStd(3) s.yStd(3) xyStd intStd];

SyntaxError: invalid syntax (<ipython-input-7-7d86aa2096b6>, line 2)

In [ ]:
# xstd is xrms0
function [fint, xmean, xstd, xvar, skew, kurt, muRaw, mu] = util_moments(x, f, dim)
%MOMENTS
%  MOMENTS(X, F, DIM) returns sum, mean, std, and var of distribution F(X)
%  binned by X. The moments are evaluated over the longest dimension if X
%  is an array, or over dimension DIM if specified.

% Input arguments:
%    X: Coordinates of distribution
%    F: Distribution F(X)
%    DIM: Dimension over which to take the moments, defaults to dimension
%         with largest size

% Output arguments:
%    FINT: Sum of F
%    XMEAN, XSTD, XVAR: mean, std, and variance of F(X)

% Compatibility: Version 7 and higher
% Called functions: none

% Author: Henrik Loos, SLAC

% --------------------------------------------------------------------
% Check input arguments
if nargin < 3
    [dummy,dim]=max(size(f)); % Find vector orientation
end

fint=sum(f,dim);
[x1sum,x2sum,x1,x2,x3,x4,skew,kurt]=deal(zeros(size(fint)));
good=fint ~= 0;

if numel(f) == numel(x)
    x=reshape(x,size(f));
    x1sum=sum(   x.*f,dim);
    x2sum=sum(x.*x.*f,dim);
    if nargout > 4
        x3sum=sum(   x.*x.*x.*f,dim);
        x4sum=sum(x.*x.*x.*x.*f,dim);
    end
elseif dim == 1
    x=reshape(x,1,[]);
    x1sum(:)=(   x)*f(:,:);
    x2sum(:)=(x.*x)*f(:,:);
    if nargout > 4
        x3sum=(   x.*x.*x)*f(:,:);
        x4sum=(x.*x.*x.*x)*f(:,:);
    end
elseif dim == 2 && size(f(:,:,:),3) == 1
    x=reshape(x,[],1);
    x1sum(:)=f*(   x);
    x2sum(:)=f*(x.*x);
    if nargout > 4
        x3sum=f*(   x.*x.*x);
        x4sum=f*(x.*x.*x.*x);
    end
else
    len=[1 1 1];len(dim)=length(x);
    x=reshape(x,len);
    x=repmat(x,size(f)./size(x));
    x1sum=sum(   x.*f,dim);
    x2sum=sum(x.*x.*f,dim);
    if nargout > 4
        x3sum=sum(   x.*x.*x.*f,dim);
        x4sum=sum(x.*x.*x.*x.*f,dim);
    end
end

x1(good)=x1sum(good)./fint(good);
x2(good)=x2sum(good)./fint(good);
xmean=x1;
xvar=x2-xmean.^2;
xstd=real(sqrt(xvar));
if nargout > 4
    x3(good)=x3sum(good)./fint(good);
    x4(good)=x4sum(good)./fint(good);
    m3=x3-3*xmean.*x2+2*xmean.^3;
    m4=x4-4*xmean.*x3+6*xmean.^2.*x2-3*xmean.^4;
    skew(good)=m3(good)./xstd(good).^3;
    kurt(good)=m4(good)./xstd(good).^4-3;
    muRaw=cat(dim,x1,x2,x3,x4);
    mu=cat(dim,zeros(size(x1)),xvar,m3,m4);
end
    